BIKE

https://www.kaggle.com/competitions/bike-sharing-demand/
```
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals
```

In [1]:
import numpy as np
import pandas as pd

from datetime import date, datetime, time, timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

# 구조 확인
```
df.head()
df.info()
df.shape
```

In [2]:
# train = pd.read_csv('./train.csv', parse_dates=['datetime']) # 날짜연습을 하기위해 오브젝으로 놔두져
train = pd.read_csv('./train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


# 컬럼명 확인
```
train.columns
```

In [3]:
# 컬럼명 변경
train = train.rename(columns={'datetime':'regdate'})
train.columns

Index(['regdate', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

In [4]:
# 컬럼명 소문자로 변경
train.columns = train.columns.str.lower()
train.columns

Index(['regdate', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

# 결측치 확인

In [5]:
# 결측치가 0보다 큰거만
train.isna().sum()[train.isna().sum()>0]

Series([], dtype: int64)

# 오브젝트 확인

In [6]:
# 오브젝트만
train.select_dtypes('object').columns

Index(['regdate'], dtype='object')

In [7]:
test_time_str = train['regdate'][0]
type(test_time_str)

str

In [8]:
datetime.strptime(test_time_str, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2011, 1, 1, 0, 0)

In [9]:
datetime.fromisoformat(test_time_str)

datetime.datetime(2011, 1, 1, 0, 0)

In [10]:
traincp = train.copy()
traincp['regdate_1'] = traincp['regdate'].apply(lambda x:datetime.fromisoformat(x) if type(x) is str else x)
traincp['regdate_1']

0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
                ...        
10881   2012-12-19 19:00:00
10882   2012-12-19 20:00:00
10883   2012-12-19 21:00:00
10884   2012-12-19 22:00:00
10885   2012-12-19 23:00:00
Name: regdate_1, Length: 10886, dtype: datetime64[ns]

In [11]:
traincp['regdate_2'] = pd.to_datetime(traincp['regdate'])
traincp['regdate_2']

0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
                ...        
10881   2012-12-19 19:00:00
10882   2012-12-19 20:00:00
10883   2012-12-19 21:00:00
10884   2012-12-19 22:00:00
10885   2012-12-19 23:00:00
Name: regdate_2, Length: 10886, dtype: datetime64[ns]

In [12]:
traincp['regdate_3'] = traincp['regdate'].astype('datetime64')
traincp['regdate_3']

0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
                ...        
10881   2012-12-19 19:00:00
10882   2012-12-19 20:00:00
10883   2012-12-19 21:00:00
10884   2012-12-19 22:00:00
10885   2012-12-19 23:00:00
Name: regdate_3, Length: 10886, dtype: datetime64[ns]

In [13]:
train['regdate'] = traincp['regdate_3']
del traincp
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   regdate     10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB


# EDA

## regdate

In [14]:
train['y'] = train['regdate'].dt.year
train['m'] = train['regdate'].dt.month
train['d'] = train['regdate'].dt.day
train['h'] = train['regdate'].dt.hour
train.head()

,regdate,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,y,m,d,h
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4


### index화, 해제
```
df.set_index(컬럼명)
df.reset_index()
```

In [20]:
train['regdate'].nunique()

10886

In [15]:
train = train.set_index('regdate')
train.index

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00',
               '2011-01-01 02:00:00', '2011-01-01 03:00:00',
               '2011-01-01 04:00:00', '2011-01-01 05:00:00',
               '2011-01-01 06:00:00', '2011-01-01 07:00:00',
               '2011-01-01 08:00:00', '2011-01-01 09:00:00',
               ...
               '2012-12-19 14:00:00', '2012-12-19 15:00:00',
               '2012-12-19 16:00:00', '2012-12-19 17:00:00',
               '2012-12-19 18:00:00', '2012-12-19 19:00:00',
               '2012-12-19 20:00:00', '2012-12-19 21:00:00',
               '2012-12-19 22:00:00', '2012-12-19 23:00:00'],
              dtype='datetime64[ns]', name='regdate', length=10886, freq=None)

## casual, registered, count

In [16]:
train[train['casual']+train['registered']!=train['count']].shape[0]

0